In [ ]:
# Шаг 1. Создаем "стриминговый" датафрейм (readStream), задаем - как часто выполнять (через каждые 1000 файлов)

streamingDataFrame = spark.readStream\
    .schema(staticSchema)\
    .option("maxFilesPerTrigger", 1000)\
    .format("csv")\
    .option("header", "true")\
    .load("/data/retail-data/by-day/*.csv")

# Шаг 2. Обрабатываем его так, как нам нужно (например, агрегируем)

purchaseByCustomerPerHour = streamingDataFrame\
    .selectExpr( "CustomerId", "(UnitPrice * Quantity) as total_cost", "InvoiceDate")\
    .groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
    .sum("total_cost")

# Шаг 3. Запускаем стриминг (в данном примере будет формироваться таблица в памяти, но это могли бы быть файлы или топики Kafka)

purchaseByCustomerPerHour.writeStream\
    .format("memory")\
    .queryName("customer_purchases")\
    .outputMode("complete")\
    .start()